In [1]:
# Установка transformers
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 31.7 MB/s 
     |████████████████████████████████| 163 kB 68.5 MB/s 
     |████████████████████████████████| 7.6 MB 60.5 MB/s 


In [2]:
#Установка catboost
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.3 MB/s 


In [3]:
import pandas as pd
from itertools import groupby

In [4]:
#import необходимых модулей

import pandas as pd
import yaml
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.utils.class_weight import compute_class_weight

import numpy as np
from datetime import datetime, timedelta

from catboost import CatBoostClassifier, Pool

In [5]:
#Считывание данных в DataFrame 
train= pd.read_csv('train_net_2_2.csv', sep=';', index_col=None)
test= pd.read_csv('test_net_2_2.csv', sep=';', index_col=None)


In [6]:
train

,team_id,id,Analytical thinking,Systemic thinking,Adaptability,Focus,game_id,period_1,period_2,period_3,...,Категория 4 оценка стокл,Категория 4 место стокл,Категория 5 средняя оценка,Категория 5 среднее место,Категория 5 оценка стокл,Категория 5 место стокл,Категория 6 средняя оценка,Категория 6 среднее место,Категория 6 оценка стокл,Категория 6 место стокл
0,1440,10884,4,4,4,4,202,Cancellation of bonus for violation of labor d...,"Contextual advertising, Advertising in print m...","Managers qualification, Anti-bureaucratic days...",...,8.225975,1.825742,18.00,2.5,8.205689,0.957427,110.75,6.0,46.743092,1.500000
1,1536,10106,5,4,5,5,211,"Development of productionвЂ™s optimisation, Bo...","Repeat inspection of batch products, Developme...","Optimizing the placement of equipment, Increas...",...,3.774917,1.500000,18.00,1.0,9.055385,2.000000,190.00,2.5,34.068558,1.707825
2,1742,10438,6,5,5,5,242,"Manufacturing staff qualification, To place pa...",Minimal fixed + % by the results of a period (...,Quality of the purchased materials Raw (carbon...,...,11.460076,1.000000,21.25,2.0,14.568802,0.500000,179.75,2.0,86.580887,2.362908
3,1272,10130,5,4,5,5,184,"Fabrication staff salaty, Analysis of the impo...","Health, Analysis of the market structure for t...","Education of the dealersвЂ™ sales staff, Quant...",...,5.795113,2.000000,18.50,1.5,10.082989,0.577350,117.50,5.0,42.280807,2.000000
4,1382,10667,4,5,5,5,193,"Pick of contractors and providers, Engineers q...","Media-advertising, The best physical propertie...","Conduct customer analysis on the market, Firef...",...,16.780445,1.500000,22.50,3.0,15.000000,0.816497,200.00,3.0,102.264363,1.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,1274,10563,5,4,5,4,184,Quality of the purchased materials Raw (carbon...,Quantity of work shifts,"Development of productionвЂ™s optimisation, De...",...,5.500000,1.632993,15.25,4.0,7.182154,0.500000,102.25,4.0,45.028695,1.000000
676,1526,10386,5,4,5,5,210,Medium strategy of staff recruitment,Bonuses for innovation and rationalization of ...,Cancellation of bonus for violation of labor d...,...,3.403430,0.500000,9.25,6.0,3.862210,2.000000,169.75,2.5,25.250413,1.892969
677,1670,10675,5,4,5,4,235,"Optimisation of the loading unitвЂ™s shape, Th...","Size of a reserve stock of MR, Size of a reser...","Size of a reserve stock of MR, The volume of o...",...,3.500000,2.217356,12.50,2.5,6.137318,1.290994,141.00,3.0,26.520432,1.825742
678,1808,10366,4,4,4,4,248,Improvement of the documentation of entrance c...,Sending staff on short-term courses of creativ...,Conducting regular medical examinations and br...,...,6.800735,1.258306,15.25,3.5,9.673848,1.707825,193.00,2.0,24.344746,0.500000


In [7]:
test

,team_id,id,game_id,period_1,period_2,period_3,period_4,Период_1_всего,Период_2_всего,Период_3_всего,...,Категория 4 оценка стокл,Категория 4 место стокл,Категория 5 средняя оценка,Категория 5 среднее место,Категория 5 оценка стокл,Категория 5 место стокл,Категория 6 средняя оценка,Категория 6 среднее место,Категория 6 оценка стокл,Категория 6 место стокл
0,1912,10199,262,"Setting the base selling price, Quantity of wo...","Additions to reserve capital, Volume discounts",Development and functioning of the companyвЂ™s...,"Hoverbikes Production line 2500 units, Setting...",2,2,3,...,6.800735,2.380476,17.25,2.5,9.639329,1.707825,154.00,3.5,46.224092,2.061553
1,1458,10539,204,Additions to reserve capital,"Pick of contractors and providers, Additional ...","[Company #1] Acquisition of the company, Metho...","Bicycles Production line 40000 units, Increase...",1,9,12,...,10.144785,2.061553,18.00,5.0,12.192894,2.217356,121.75,4.5,62.082606,2.160247
2,1348,10174,190,Not decision,Improvement of the technology of entrance cont...,Establishment of mid-term contract - Industria...,"Quality of the purchased materials Lubricants""...",0,28,21,...,17.416467,0.957427,18.50,2.0,10.847427,1.500000,239.50,1.5,71.649610,0.957427
3,1760,10465,244,Improvement of the technology of entrance cont...,R&D enhancement of assortment qualities Standa...,"To sell overdue accounts receivable, which do ...","Research on elasticity of market demand, [long...",15,18,29,...,7.937254,1.892969,18.75,3.0,10.111874,0.816497,135.75,3.5,37.897889,1.707825
4,1260,10066,183,"Quantity of work shifts, Keep up the aesthetic...",Bonuses for the fulfillment of the production ...,"Development of productionвЂ™s optimisation, Qu...","Enguneers salary, Stepwise staff numerosityвЂ™...",7,23,16,...,6.849574,1.892969,16.25,2.5,7.365460,0.957427,79.75,5.5,76.621907,1.892969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,1562,10433,214,"Pick of contractors and providers, Value of pr...",Analysis of the importance of product properti...,Quantity of work shifts,Not decision,15,6,1,...,15.545632,1.414214,20.50,3.0,13.279056,1.154701,221.75,2.5,85.737487,2.061553
289,1278,10893,184,R&D enhancement of assortment qualities Standa...,"Discipline, Moral qualities, Independently hir...",Collection information and statistics of the d...,"Creativity, Managers salary, Discipline, Hirin...",14,52,34,...,15.607156,0.957427,16.50,2.5,10.630146,2.160247,191.75,2.0,85.445402,0.816497
290,1474,10909,206,Not decision,Not decision,Not decision,The volume of of transportation - Worldwide co...,0,0,0,...,3.109126,1.500000,18.50,3.5,11.474610,0.577350,172.25,2.5,13.720423,1.290994
291,1268,10889,184,Renewal of checkpoints in production and stora...,"Security alarm upgrade, Evaluation of the char...","Advertising in print media, Radio advertising,...",Analysis of the product satisfaction,6,2,5,...,14.271183,1.707825,10.25,5.5,2.217356,0.577350,211.50,2.5,87.401373,1.707825


In [8]:
train.columns

Index(['team_id', 'id', 'Analytical thinking', 'Systemic thinking',
       'Adaptability', 'Focus', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4', 'Период_1_всего', 'Период_2_всего', 'Период_3_всего',
       'Период_4_всего', 'TestSummAll', 'Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz',
       'Категория 1 средняя оценка', 'Категория 1 среднее место',
       'Категория 1 оценка стокл', 'Категория 1 место стокл',
       'Категория 2 оценка', 'Категория 2 среднее место',
       'Категория 2 оценка стокл', 'Категория 2 место стокл',
       'Категория 3 средняя оценка', 'Категория 3 среднее место',
       'Категория 3 оценка стокл', 'Категория 3 место стокл',
       'Категория 4 средняя оценка', 'Категория 4 среднее место',
       'Категория 4 оценка стокл', 'Категория 4 место стокл',
       'Категория 5 средняя оценка', 'Категория 5 среднее место',
       'Категория 5 оценка стокл', 'Категория 5 место стокл',


In [9]:
# выбираем поля для нейросети СatBoost по бинарной классификации

X_train=train[['team_id', 'game_id',  'Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz','Период_1_всего', 'Период_2_всего',
       'Период_3_всего', 'Период_4_всего',  'TestSummAll',
        'Категория 1 средняя оценка', 'Категория 1 среднее место',
       'Категория 2 оценка', 'Категория 2 среднее место',
       'Категория 3 средняя оценка', 'Категория 3 среднее место',
       'Категория 4 средняя оценка', 'Категория 4 среднее место',
       'Категория 5 средняя оценка', 'Категория 5 среднее место',
       'Категория 6 средняя оценка', 'Категория 6 среднее место']]
y_train=train[['Systemic thinking']]


testnet=test[['team_id', 'game_id',  'Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz','Период_1_всего', 'Период_2_всего',
       'Период_3_всего', 'TestSummAll', 'Период_4_всего',
        'Категория 1 средняя оценка', 'Категория 1 среднее место',
       'Категория 2 оценка', 'Категория 2 среднее место',
       'Категория 3 средняя оценка', 'Категория 3 среднее место',
       'Категория 4 средняя оценка', 'Категория 4 среднее место',
       'Категория 5 средняя оценка', 'Категория 5 среднее место',
       'Категория 6 средняя оценка', 'Категория 6 среднее место']]

In [10]:
train.columns

Index(['team_id', 'id', 'Analytical thinking', 'Systemic thinking',
       'Adaptability', 'Focus', 'game_id', 'period_1', 'period_2', 'period_3',
       'period_4', 'Период_1_всего', 'Период_2_всего', 'Период_3_всего',
       'Период_4_всего', 'TestSummAll', 'Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz',
       'Категория 1 средняя оценка', 'Категория 1 среднее место',
       'Категория 1 оценка стокл', 'Категория 1 место стокл',
       'Категория 2 оценка', 'Категория 2 среднее место',
       'Категория 2 оценка стокл', 'Категория 2 место стокл',
       'Категория 3 средняя оценка', 'Категория 3 среднее место',
       'Категория 3 оценка стокл', 'Категория 3 место стокл',
       'Категория 4 средняя оценка', 'Категория 4 среднее место',
       'Категория 4 оценка стокл', 'Категория 4 место стокл',
       'Категория 5 средняя оценка', 'Категория 5 среднее место',
       'Категория 5 оценка стокл', 'Категория 5 место стокл',


In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 680 entries, 0 to 679
Data columns (total 23 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   team_id                      680 non-null    int64  
 1   game_id                      680 non-null    int64  
 2   Analytical thinking_prognoz  680 non-null    int64  
 3   Systemic thinking_prognoz    680 non-null    int64  
 4   Adaptability_prognoz         680 non-null    int64  
 5   Focus_prognoz                680 non-null    int64  
 6   Период_1_всего               680 non-null    int64  
 7   Период_2_всего               680 non-null    int64  
 8   Период_3_всего               680 non-null    int64  
 9   Период_4_всего               680 non-null    int64  
 10  TestSummAll                  680 non-null    object 
 11  Категория 1 средняя оценка   680 non-null    float64
 12  Категория 1 среднее место    680 non-null    float64
 13  Категория 2 оценка  

In [12]:
# Определяем веса для классов кластеров
classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes = np.unique(y_train), y = np.ravel(y_train))
class_weights = dict(zip(classes, weights))
class_weights

{1: 34.0,
 3: 2.6666666666666665,
 4: 0.2769857433808554,
 5: 1.096774193548387,
 6: 13.6}

In [13]:
#Обучаем 

pool_train_solution = Pool(X_train, y_train, cat_features = ['team_id', 'game_id','Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz'],text_features = [ 'TestSummAll'])
pool_test_solution = Pool(testnet,cat_features = ['team_id', 'game_id','Analytical thinking_prognoz',
       'Systemic thinking_prognoz', 'Adaptability_prognoz', 'Focus_prognoz'],text_features = [  'TestSummAll'])

model_test = CatBoostClassifier(iterations=25,  task_type='GPU',depth=12, verbose =True, random_seed=13,# learning_rate=0.015,
                               leaf_estimation_method= 'Gradient',bootstrap_type= 'Poisson',objective = 'MultiClass',eval_metric = 'Accuracy',loss_function='Recall', class_weights=class_weights)
model_test.fit(pool_train_solution)

0:	learn: 0.9177577	total: 328ms	remaining: 7.87s
1:	learn: 0.9237323	total: 642ms	remaining: 7.39s
2:	learn: 0.9431542	total: 879ms	remaining: 6.44s
3:	learn: 0.9463964	total: 1.09s	remaining: 5.74s
4:	learn: 0.9443926	total: 1.29s	remaining: 5.16s
5:	learn: 0.9439853	total: 1.51s	remaining: 4.8s
6:	learn: 0.9451909	total: 1.72s	remaining: 4.42s
7:	learn: 0.9480258	total: 1.92s	remaining: 4.07s
8:	learn: 0.9456146	total: 2.11s	remaining: 3.75s
9:	learn: 0.9456146	total: 2.32s	remaining: 3.48s
10:	learn: 0.9504533	total: 2.51s	remaining: 3.2s
11:	learn: 0.9488404	total: 2.78s	remaining: 3.02s
12:	learn: 0.9460055	total: 3.03s	remaining: 2.8s
13:	learn: 0.9488404	total: 3.22s	remaining: 2.53s
14:	learn: 0.9480258	total: 3.41s	remaining: 2.27s
15:	learn: 0.9480258	total: 3.62s	remaining: 2.03s
16:	learn: 0.9520662	total: 3.83s	remaining: 1.8s
17:	learn: 0.9508442	total: 4.03s	remaining: 1.57s
18:	learn: 0.9492478	total: 4.23s	remaining: 1.33s
19:	learn: 0.9492478	total: 4.43s	remaining: 

In [14]:
# предварительный тест схождения на всем обучающем множестве
y_pred = model_test.predict(X_train)


In [15]:
print("Значение метрики recall на трейне : ", recall_score(y_train, y_pred, average='macro', zero_division=True))

Значение метрики recall на трейне :  0.9395539057880559


In [16]:
#Получение ответов
y_pred_solution = model_test.predict(pool_test_solution)

In [17]:
# Посмотрим предварительный прогноз
test['Systemic thinking_prognoz'].value_counts()

4    140
5     93
3     54
1      4
6      2
Name: Systemic thinking_prognoz, dtype: int64

In [18]:
# Заводим результаты в тест
test['Systemic thinking']=y_pred_solution
test['Systemic thinking'].value_counts()

4    145
5     92
3     51
1      3
6      2
Name: Systemic thinking, dtype: int64

In [19]:
test_itog=test[['id','Systemic thinking']]
test_itog

,id,Systemic thinking
0,10199,4
1,10539,4
2,10174,5
3,10465,5
4,10066,4
...,...,...
288,10433,5
289,10893,5
290,10909,4
291,10889,3


In [20]:
test_itog.to_csv('прогноз по Systemic thinking предварительный второй этап 13 24 с текстами.csv', sep=',', index=None)